In [2]:
import pandas as pd
import numpy as np
import warnings
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

# func

In [40]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:[float(element) for element in x])
  tp_data['state_day_start'] = tp_data['state_day_start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tp_data['state_day_end'] = tp_data['state_day_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))

  spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
  spread_df['spread'] = spread_df['spread'] / 2
  s = np.setdiff1d(tp_data.state_ticker.to_numpy(), spread_df.ticker.to_numpy())
  tp_data['spread'] = tp_data.state_ticker.loc[~tp_data.state_ticker.isin(s)].apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
  tp_data['spread'].loc[tp_data.state_ticker.isin(s)] = 5
  tp_data['deviation'] = 1
  tp_data['point'] = tp_data.state_ticker.loc[~tp_data.state_ticker.isin(s)].apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
  tp_data['point'].loc[tp_data.state_ticker.isin(s)] = 0.01
  tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
  tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput0.csv')

In [111]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [70]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max())
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

# TP data

In [8]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput270.csv')
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-05-16 13:50:00,2022-05-18 13:50:00,90.985,90.510,NaN,7.400000,3.610526,3.884211,3.610526,"[-0.13684210526312074, -0.26315789473680745, -..."
1,MRK,2,2022-05-18 18:50:00,2022-05-19 19:00:00,92.204,92.394,NaN,5.915789,NaN,NaN,-1.000000,"[0.2842105263158036, 0.6526315789474258, 1.178..."
2,MRK,1,2022-05-25 16:10:00,2022-05-26 14:20:00,93.138,92.818,NaN,2.225000,0.131250,NaN,-1.000000,"[0.44374999999999665, 0.4750000000000133, 0.13..."
3,MRK,2,2022-05-26 14:35:00,2022-06-01 19:35:00,93.028,93.458,NaN,6.902326,3.600000,5.506977,3.600000,"[-0.05116279069765351, -0.5860465116278868, -0..."
4,MRK,2,2022-06-01 19:30:00,2022-06-03 13:40:00,91.385,92.145,NaN,3.480263,-0.125000,NaN,1.388158,"[-0.006578947368433847, -0.1250000000000187, 0..."


In [374]:
a = None
if a:
    print('a')

In [371]:
data1 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
data2 = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread2.csv')
data1['spread2'] = data1.ticker.apply(lambda x: data2.spread.loc[data2.ticker == x].iloc[0])
data1.loc[data1.spread != data1.spread2]

,Unnamed: 0,ticker,point,spread,deviation,spread2
0,0,AAPL,0.01,6,5,9
5,5,AMD,0.01,10,5,25
8,8,AMZN,0.01,4,5,6
12,12,BAC,0.01,8,5,9
13,13,BA,0.01,24,5,46
24,24,DE30,0.10,39,5,36
28,28,FB,0.01,8,5,41
38,38,JP225,0.10,90,5,89
50,50,MSFT,0.01,10,5,21
52,52,NFLX,0.01,24,5,81


In [7]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,634.000000,634.000000,634.000000,634.000000,634.000000
mean,0.133611,2.637146,0.020554,0.076891,-0.011929
std,3.938061,5.902176,3.700856,3.054069,3.709052
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.035287,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.538088,-1.000000,-1.000000,-1.000000
75%,-1.000000,2.808854,-1.000000,-1.000000,-1.000000
max,38.100000,52.037500,38.100000,33.528571,38.100000


In [260]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     46.891751
state_max_r             985.999242
state_MA_r               -9.228510
state_RSI_r              20.969008
last_r                  -16.195593
dtype: float64

In [266]:
tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
tp_data.estimate_with_thresh.sum()

-63.334356909509566

In [264]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."


# concat data

In [72]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/*.csv"):
    tp_data = preprocess_tp_data(path)
    tp_data['bash_path'] = str(os.path.basename(path))
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)


In [143]:
cut_thresh = 0.61
concat_data['estimate_with_thresh'] = concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 6 , cut_thresh) )
concat_data['estimate_with_thresh2'] = concat_data.state_chain_new.apply(lambda x : x.max() * cut_thresh )
concat_data.estimate_with_thresh2.loc[concat_data.estimate_with_thresh2 < 3] = -1
concat_data.loc[concat_data.state_ticker.isin(portfolio)].estimate_with_thresh.sum()

545.9006094719389

In [144]:
stock_tp_data = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].groupby(['state_ticker' , 'bash_path']).sum()
stock_tp_data['win_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r > 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data['loss_count'] = concat_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2', 'bash_path']].loc[concat_data.state_MA_r < 0].groupby(['state_ticker' , 'bash_path']).count().state_MA_r
stock_tp_data.win_count.fillna(0 , inplace = True)
stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 
stock_tp_data

state_close_estimate  state_max_r  \
state_ticker bash_path                                               
AAPL         stateoutput0.csv              -14.000000    14.001688   
             stateoutput120.csv             -0.473267    24.204790   
             stateoutput150.csv             -2.117733    21.692807   
             stateoutput180.csv             -4.709986    58.150310   
             stateoutput210.csv            -13.768421    27.340992   
...                                               ...          ...   
XOM          stateoutput30.csv              -3.744828     8.060589   
YUMC         stateoutput0.csv               -3.368750     5.113889   
             stateoutput30.csv              -6.000000    11.413559   
ZTO          stateoutput0.csv               -0.519298    17.797734   
             stateoutput30.csv              -7.000000    13.482468   

                                 state_MA_r  state_RSI_r     last_r  \
state_ticker bash_path                                                
AAPL         stateoutput0.csv    -14.000000    -7.166667 -14.000000   
             stateoutput120.csv   -0.473267     2.783911  -0.473267   
             stateoutput150.csv   -1.678667    -2.117733  -2.117733   
             stateoutput180.csv    3.864214     2.939569   2.939569   
             stateoutput210.csv  -11.968672   -10.012171 -13.768421   
...                                     ...          ...        ...   
XOM          stateoutput30.csv    -2.836933    -3.744828  -3.744828   
YUMC         stateoutput0.csv     -2.553365    -3.368750  -3.368750   
             stateoutput30.csv    -6.000000    -6.000000  -6.000000   
ZTO          stateoutput0.csv     -0.519298    -0.519298  -0.519298   
             stateoutput30.csv    -6.700000    -4.218182  -7.000000   

                                 estimate_with_thresh  estimate_with_thresh2  \
state_ticker bash_path                                                         
AAPL         stateoutput0.csv              -14.000000             -14.000000   
             stateoutput120.csv              1.792516               1.792516   
             stateoutput150.csv             -2.304192              -0.352847   
             stateoutput180.csv              3.170835               5.643230   
             stateoutput210.csv             -9.011765              -3.152352   
...                                               ...                    ...   
XOM          stateoutput30.csv              -2.019314              -2.019314   
YUMC         stateoutput0.csv               -3.468421              -5.000000   
             stateoutput30.csv              -6.000000              -6.000000   
ZTO          stateoutput0.csv               -1.375346              -3.323733   
             stateoutput30.csv              -8.000000              -8.000000   

                                 win_count  loss_count  
state_ticker bash_path                                  
AAPL         stateoutput0.csv            0          14  
             stateoutput120.csv          2           8  
             stateoutput150.csv          2          10  
             stateoutput180.csv          2          15  
             stateoutput210.csv          1          16  
...                                    ...         ...  
XOM          stateoutput30.csv           1           6  
YUMC         stateoutput0.csv            1           4  
             stateoutput30.csv           0           6  
ZTO          stateoutput0.csv            2           6  
             stateoutput30.csv           0           7  

[669 rows x 9 columns]

In [140]:
portfolio = ['ADBE', 'ADP', 'AMD', 'AMGN', 'AMT', 'AMZN', 'AVGO', 'BMY', 'CHTR',
       'CMCSA', 'CME', 'COST', 'CSX', 'EA', 'GILD', 'HD', 'IBM', 'INTU',
       'KO', 'LIN', 'LMT', 'MA', 'MCD', 'MDLZ', 'MMM', 'MO', 'MRK', 'MS',
       'MSFT', 'NKE', 'PM', 'TMO', 'TSLA']

In [133]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].groupby('state_ticker').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,,,,,,,,,
ADBE,245.733843,454.073596,219.542219,50.464330,217.135973,210.339629,162.676768,12,71
ADP,19.371441,182.837133,10.269597,-4.180440,11.825427,12.450850,13.364826,12,62
AMD,-1.792136,304.022760,28.367227,29.583851,20.540838,-8.145036,-14.294156,22,108
AMGN,88.005697,290.336531,71.197741,51.462124,71.784292,67.518925,67.237293,18,60
AMT,-6.532756,140.338333,-24.747657,1.560624,-25.230010,-10.969843,3.041998,12,53
AMZN,6.534663,255.557605,-3.698607,8.246813,-8.158632,-3.869866,0.124077,12,95
AVGO,20.056106,197.680921,10.609004,19.712367,6.324333,24.781233,22.317700,11,59
BMY,30.929567,246.899391,-1.484876,17.841006,14.667181,12.636781,2.670658,19,72
CHTR,29.969377,277.617605,132.301232,34.030915,129.855632,23.828356,85.923770,13,54


In [125]:
min_thresh_list = np.arange(6.0,15.0,0.5)
cut_thresh_list =np.arange(0.5,0.8,0.01)
grid_list = []
for i in min_thresh_list:
    for j in cut_thresh_list:
        grid_list.append([i,j])
max = 0
i_temp = 0
j_temp = 0
for x in grid_list:
    temp = get_return(x[0],x[1])
    if(temp > max):
        max = temp
        i_temp = x[0]
        j_temp = x[1]

print(max, i_temp, j_temp)

545.9006094719391 6.0 0.6100000000000001


In [120]:
def get_return(min_thresh, cut_thresh):
    sub_concat_data.estimate_with_thresh = sub_concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , min_thresh , cut_thresh) )
    return sub_concat_data.estimate_with_thresh.sum()

get_return(8,0.6)

508.18896510196464

In [112]:
sub_concat_data = concat_data.loc[(concat_data.state_ticker.isin(portfolio))]

min_thresh = 8
cut_thresh = 0.7
sub_concat_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , min_thresh , cut_thresh) )
sub_concat_data

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,...,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new,bash_path,estimate_with_thresh,estimate_with_thresh2
0,0,MRK,1,2023-01-13 20:10:00,2023-01-17 14:55:00,111.746,111.656,-1.0,4.711111,-1.0,...,-1.0,"[0.3777777777778304, 1.7111111111111637, 1.6, ...",3.5,1,0.01,1.500000,"[0.25185185185187686, 1.1407407407407302, 1.06...",stateoutput30.csv,-1.0,-1.0
1,1,MRK,2,2023-01-19 14:45:00,2023-01-19 15:55:00,109.264,109.404,-1.0,2.885714,-1.0,...,-1.0,"[0.3857142857143379, 0.6714285714285946, 1.885...",3.5,1,0.01,1.321429,"[0.2918918918919245, 0.5081081081081136, 1.427...",stateoutput30.csv,-1.0,-1.0
2,2,MRK,1,2023-01-23 14:40:00,2023-01-24 14:40:00,109.390,109.140,-1.0,3.560000,-1.0,...,-1.0,"[-0.12000000000000455, -0.39999999999997726, -...",3.5,1,0.01,1.180000,"[-0.10169491525424115, -0.33898305084743835, -...",stateoutput30.csv,-1.0,-1.0
3,3,MRK,1,2023-01-27 14:40:00,2023-01-27 14:45:00,106.480,106.180,-1.0,-1.000000,-1.0,...,-1.0,"[-1.0333333333333508, -1.166666666666706]",3.5,1,0.01,1.150000,"[-0.8985507246376951, -1.0144927536232213]",stateoutput30.csv,-1.0,-1.0
4,4,MRK,1,2023-01-31 14:45:00,2023-02-02 14:40:00,106.172,105.942,-1.0,4.860870,-1.0,...,-1.0,"[0.16521739130433064, -0.39999999999998764, -0...",3.5,1,0.01,1.195652,"[0.1381818181818042, -0.33454545454544515, -0....",stateoutput30.csv,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,675,COST,1,2022-05-25 19:10:00,2022-05-25 19:25:00,441.742,440.312,-1.0,-0.036364,-1.0,...,-1.0,"[-0.036363636363650816, -0.44195804195804334, ...",9.5,1,0.01,1.073427,"[-0.03387622149838481, -0.41172638436482223, -...",stateoutput270.csv,-1.0,-1.0
6673,676,COST,2,2022-06-01 14:20:00,2022-06-02 15:00:00,464.566,466.376,-1.0,4.146961,-1.0,...,-1.0,"[-0.10718232044196671, 0.12486187845306942, 0....",9.5,1,0.01,1.058011,"[-0.1013054830286996, 0.11801566579637371, 0.5...",stateoutput270.csv,-1.0,-1.0
6674,677,COST,1,2022-06-03 17:10:00,2022-06-03 17:15:00,481.792,479.862,-1.0,-0.104663,-1.0,...,-1.0,"[-0.10466321243523184, -0.995854922279773]",9.5,1,0.01,1.054404,"[-0.09926289926289789, -0.9444717444717246]",stateoutput270.csv,-1.0,-1.0
6675,678,COST,2,2022-06-06 15:45:00,2022-06-09 13:45:00,475.034,477.374,-1.0,4.877778,-1.0,...,-1.0,"[0.10427350427349061, -0.023931623931616415, -...",9.5,1,0.01,1.044872,"[0.09979550102248187, -0.022903885480565416, -...",stateoutput270.csv,-1.0,-1.0


In [670]:
portfolio_list = stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10].index.get_level_values(0).unique()

In [49]:
test_df = stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio_list)]
x = test_df.loc[(test_df.estimate_with_thresh < -10)].groupby('state_ticker').count()
month_high_lost = x.loc[x.estimate_with_thresh > 2].index
new_portfolio_list = np.setdiff1d(portfolio_list, month_high_lost)

In [50]:
new_portfolio_list

array(['ABT', 'ADBE', 'ADP', 'AMC', 'AMD', 'AMGN', 'AMZN', 'AVGO', 'BBBY',
       'BIDU', 'BILI', 'BMY', 'BRQS', 'BYND', 'CAN', 'CHTR', 'CME',
       'COST', 'CSCO', 'DE30', 'EA', 'EDU', 'EQIX', 'FUTU', 'GILD', 'HD',
       'IBM', 'INTU', 'JP225', 'KO', 'LLY', 'LMT', 'MA', 'MCD', 'MDLZ',
       'META', 'MMM', 'MRK', 'NKE', 'PM', 'TME', 'TMO', 'TSLA', 'TSM',
       'UPS', 'VIPS', 'WFC', 'XOM', 'YUMC'], dtype=object)

In [108]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].sum().estimate_with_thresh

477.86756914776913

In [65]:
np.setdiff1d(concat_data.loc[(concat_data.bash_path == 'stateoutput0.csv')].state_ticker.unique() , np.array(portfolio))

array(['AAPL', 'ABBV', 'ABT', 'AMC', 'ATVI', 'BABA', 'BAC', 'BBBY',
       'BEKE', 'BIDU', 'BILI', 'BRQS', 'BYND', 'CAN', 'CVS', 'DE30',
       'DKKHUF', 'DKKSGD', 'DKKZAR', 'DXY', 'EDU', 'F', 'FTNT', 'FUTU',
       'GOOGL', 'INTC', 'JNJ', 'JP225', 'JPM', 'LLY', 'META', 'NFLX',
       'NTES', 'ORCL', 'PEP', 'PFE', 'PG', 'PYPL', 'REGN', 'T', 'TIGR',
       'TME', 'TSM', 'UPS', 'VIPS', 'VRTX', 'WFC', 'WMT', 'XOM', 'YUMC',
       'ZTO'], dtype=object)

In [68]:
concat_data.loc[(concat_data.bash_path == 'stateoutput0.csv') & (concat_data.state_ticker.isin(portfolio))].groupby('state_ticker').sum()

,index,state_order,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,spread,deviation,point,sum_gap,estimate_with_thresh,estimate_with_thresh2
state_ticker,,,,,,,,,,,,,,,
ADBE,2160,5,1794.611,1786.437,-5.000000,3.884320,-4.871761,-5.000000,-5.000000,37.5,5,0.05,5.286744,-5.000000,-5.000000
ADP,1638,11,1568.606,1568.966,-7.000000,6.061252,-6.754839,-7.000000,-7.000000,35.0,7,0.07,8.089961,-7.000000,-7.000000
AMD,7140,26,1180.705,1183.180,-15.000000,9.468912,-15.000000,-15.000000,-15.000000,187.5,15,0.15,21.211194,-15.000000,-15.000000
AMGN,1837,22,2601.979,2607.494,-1.439556,18.990263,-2.548148,-2.548148,-2.548148,60.5,11,0.11,12.955058,-5.017313,0.504478
AVGO,3579,7,3526.180,3523.330,-3.622222,-0.286229,-3.622222,-3.622222,-3.622222,99.0,6,0.06,7.116585,-3.867005,-6.000000
BMY,3289,20,923.327,923.732,-9.185714,16.010420,-8.919048,-2.006924,-9.185714,39.0,13,0.13,17.181345,-9.537500,-13.000000
CMCSA,1839,10,231.484,231.774,-6.000000,3.512900,-6.000000,-6.000000,-6.000000,15.0,6,0.06,7.933083,-6.000000,-6.000000
GILD,1095,15,837.011,837.388,-10.000000,7.662626,-10.000000,-10.000000,-10.000000,30.0,10,0.10,12.250916,-10.000000,-10.000000
IBM,252,13,1066.847,1067.412,26.171429,45.620237,26.171429,18.742857,26.171429,28.0,8,0.08,9.350930,18.102703,18.597297


In [128]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].groupby('bash_path').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
bash_path,,,,,,,,,
stateoutput0.csv,-56.918769,245.468284,-70.289647,-70.699238,-75.474362,-74.927977,-67.868249,13,145
stateoutput120.csv,67.010320,733.490306,35.511833,68.329262,42.518169,56.676969,32.151016,41,234
stateoutput150.csv,402.871446,1234.015344,500.310076,95.060175,497.769482,279.389716,290.804150,55,214
stateoutput180.csv,84.595448,849.202777,82.074267,74.783687,78.416341,30.204824,76.096967,58,231
stateoutput210.csv,117.525167,794.241566,79.947217,185.254151,77.452274,101.000270,112.554390,47,222
stateoutput240.csv,-0.238627,558.982311,-58.316654,2.663700,-60.817176,-19.090447,-11.741781,33,226
stateoutput270.csv,77.238673,724.529664,51.350412,71.471736,42.034867,57.360287,21.174192,51,241
stateoutput30.csv,-10.784895,353.357832,26.574457,-55.795796,29.153486,-19.071525,-13.050012,16,122
stateoutput60.csv,118.747117,806.204240,47.642939,93.317948,35.723780,83.467001,63.031739,56,253


In [129]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0).isin(portfolio)].groupby('state_ticker').sum()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,estimate_with_thresh,estimate_with_thresh2,win_count,loss_count
state_ticker,,,,,,,,,
ADBE,245.733843,454.073596,219.542219,50.464330,217.135973,194.365300,162.676768,12,71
ADP,19.371441,182.837133,10.269597,-4.180440,11.825427,13.260849,13.364826,12,62
AMD,-1.792136,304.022760,28.367227,29.583851,20.540838,1.872659,-14.294156,22,108
AMGN,88.005697,290.336531,71.197741,51.462124,71.784292,69.667294,67.237293,18,60
AMT,-6.532756,140.338333,-24.747657,1.560624,-25.230010,-5.262478,3.041998,12,53
AMZN,6.534663,255.557605,-3.698607,8.246813,-8.158632,4.527988,0.124077,12,95
AVGO,20.056106,197.680921,10.609004,19.712367,6.324333,31.415537,22.317700,11,59
BMY,30.929567,246.899391,-1.484876,17.841006,14.667181,4.175253,2.670658,19,72
CHTR,29.969377,277.617605,132.301232,34.030915,129.855632,13.199312,85.923770,13,54


In [682]:
stock_tp_data.loc[stock_tp_data.index.get_level_values(0) == 'AMGN']

state_close_estimate  state_max_r  \
state_ticker bash_path                                               
AMGN         stateoutput120.csv             23.451317    49.252260   
             stateoutput150.csv             39.468575    81.291867   
             stateoutput180.csv             -2.117927    14.941608   
             stateoutput210.csv             -9.000000    12.679169   
             stateoutput240.csv              9.437779    25.516933   
             stateoutput270.csv             -3.642188    13.776901   
             stateoutput60.csv               9.424121    25.733684   
             stateoutput90.csv               5.099438    13.875450   

                                 state_MA_r  state_RSI_r     last_r  \
state_ticker bash_path                                                
AMGN         stateoutput120.csv   20.066773    21.327219  20.200701   
             stateoutput150.csv   34.906407    30.189320  34.030450   
             stateoutput180.csv   -1.540654    -2.117927  -2.117927   
             stateoutput210.csv   -9.000000    -9.000000  -9.000000   
             stateoutput240.csv    5.459255     3.627033   5.872520   
             stateoutput270.csv   -3.428551    -3.642188  -3.642188   
             stateoutput60.csv     7.717897     9.424121   9.424121   
             stateoutput90.csv     2.240625     2.240625   2.240625   

                                 estimate_with_thresh  estimate_with_thresh2  \
state_ticker bash_path                                                         
AMGN         stateoutput120.csv             17.638724              22.184575   
             stateoutput150.csv             33.589601              37.060699   
             stateoutput180.csv             -1.053457              -1.045769   
             stateoutput210.csv             -9.000000              -2.641379   
             stateoutput240.csv              8.380042              10.130760   
             stateoutput270.csv             -3.821723              -2.366292   
             stateoutput60.csv               8.947216              15.157700   
             stateoutput90.csv               3.780432               5.475540   

                                 win_count  loss_count  
state_ticker bash_path                                  
AMGN         stateoutput120.csv          3           6  
             stateoutput150.csv          4           4  
             stateoutput180.csv          2           7  
             stateoutput210.csv          0           9  
             stateoutput240.csv          2           5  
             stateoutput270.csv          1           8  
             stateoutput60.csv           3           2  
             stateoutput90.csv           1           3

In [ ]:
new_portfolio_list

In [705]:
spread_df = pd.read_csv('spread.csv')
spread_df.loc[spread_df.ticker.isin(new_portfolio_list)]
spread_df2 = pd.read_csv('spread2.csv')
spread_df['spred2'] = spread_df.ticker.apply(lambda x: spread_df2.spread.loc[spread_df2.ticker == x].iloc[0])
spread_df.head(50)

,Unnamed: 0,ticker,point,spread,deviation,spred2
0,0,AAPL,0.01,6,5,9
1,1,ABBV,0.01,7,5,7
2,2,ABT,0.01,7,5,7
3,3,ADBE,0.01,15,5,15
4,4,ADP,0.01,10,5,10
5,5,AMD,0.01,10,5,25
6,6,AMGN,0.01,11,5,11
7,7,AMT,0.01,10,5,10
8,8,AMZN,0.01,4,5,6
9,9,ATVI,0.01,7,5,7


# Stock_tp_data

In [46]:
def get_stock_data(tp_data):
    tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.6) )
    tp_data['estimate_with_thresh2'] = tp_data.state_chain_new.apply(lambda x : x.max() * 0.75)
    tp_data['estimate_with_thresh2'].loc[tp_data['estimate_with_thresh2'] < 4] = -1
    tp_data.estimate_with_thresh2.loc[tp_data.estimate_with_thresh2 < 5] = -1
    stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].groupby('state_ticker').sum()
    stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r', 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r' , 'estimate_with_thresh', 'estimate_with_thresh2']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
    stock_tp_data.win_count.fillna(0 , inplace = True)
    stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 

    return stock_tp_data

In [47]:
stock_tp_data = get_stock_data(concat_data)
stock_tp_data.sum()

state_close_estimate        19.951226
state_max_r              15806.710344
state_MA_r                -374.072617
state_RSI_r               -143.036494
last_r                    -491.838738
estimate_with_thresh      -967.849670
estimate_with_thresh2     -546.276837
win_count                  981.000000
loss_count                5697.000000
dtype: float64

In [48]:
portfolio_list = stock_tp_data.loc[stock_tp_data.estimate_with_thresh > -10].index

In [615]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput150.csv'
tp_data = preprocess_tp_data(path)
#tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 15 , 0.6) )
stock_tp_data = get_stock_data(tp_data)
stock_tp_data.sum()

state_close_estimate      255.321413
state_max_r              2018.145404
state_MA_r                322.612335
state_RSI_r                 3.843986
last_r                    321.780874
estimate_with_thresh      152.794267
estimate_with_thresh2     274.091012
win_count                 103.000000
loss_count                579.000000
dtype: float64

In [105]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7, cut_max_default = 7):
    c = np.array(state_return_list).copy()
    x_value = c[c > min_thresh]

    if len(x_value) > 0:
        x_value = x_value[0]
        x_index, = np.where(c == x_value)
        x = c[x_index[0]:]
        list_temp = [x[0] * cut_thresh]
        list_temp2 = [x[0] * (1 - cut_thresh)]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            temp2 = x[0:i].max() * (1 - cut_thresh)
            list_temp.append(temp)
            list_temp2.append(temp2)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        temp_df['c'] = list_temp2
        temp_df.b.loc[temp_df.c > cut_max_default] =  temp_df.a - cut_max_default
        # print(temp_df.iloc[50:]
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].b
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r


In [7]:
a = {'a':1}
'a' in a.keys() 

True

In [88]:
path = '/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput150.csv'
tp_data = preprocess_tp_data(path)

tp_data['estimate_with_thresh'] = tp_data.state_chain_new.apply(lambda x : get_return_with_max_estimate(x , 8 , 0.8) )
a = tp_data.loc[(tp_data.estimate_with_thresh > 30)].iloc[2]
a.state_chain_new , a.estimate_with_thresh , a.state_chain_new.max() * 0.8

(array([ 0.49474535,  0.97978981,  1.37590946,  2.07113985,  1.65885206,
         2.087308  ,  2.8148747 ,  3.76879547,  3.59902991,  4.18916734,
         2.87954729,  2.8148747 ,  3.04931285,  2.02263541,  2.63702506,
         2.45109135,  2.68552951,  2.60468876,  2.97655618,  3.71220695,
         2.58043654,  1.49717057,  2.00646726,  2.16006467,  1.43249798,
         2.00646726,  2.41875505,  1.24656427,  1.09296686,  1.62651576,
         1.82053355,  2.28940986,  2.66936136,  2.26515764,  1.87712207,
         1.92562652,  1.57801132,  1.61034762,  2.61277284,  2.51576395,
         2.04688763,  1.68310428,  1.09296686,  1.47291835,  1.4890865 ,
         0.69684721,  0.09054163,  0.33306386,  0.1956346 ,  0.61600647,
         0.51899757,  0.68876314,  0.72918351,  0.94745352,  1.27890057,
         1.38399353,  1.64268391,  1.90945837,  2.04688763,  3.1301536 ,
         2.8148747 ,  2.93613581,  3.38884398,  2.97655618,  2.79062247,
         2.75828618,  2.63702506,  2.66936136,  2.6

In [249]:
#tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
def get_return_with_max_estimate(state_return_list, min_thresh = 12, cut_thresh = 0.66):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

tp_data['new_estimate'] = tp_data.state_chain.apply(lambda x: get_return_with_max_estimate(x, 12, 0.66))
tp_data.new_estimate.sum()

86.98738404481048

In [152]:
tp_data.state_close_estimate.sum()

39.52315681719646

In [214]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')


In [212]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new/state_output_new/stateoutput90.csv')

In [224]:
spread_df = pd.read_csv('/Users/tranthong/Desktop/stock-app/spread.csv')
tp_data['spread'] = tp_data.state_ticker.apply(lambda x: spread_df.spread.loc[spread_df.ticker == x].iloc[0])
tp_data['deviation'] = tp_data.state_ticker.apply(lambda x: spread_df.deviation.loc[spread_df.ticker == x].iloc[0])
tp_data['point'] = tp_data.state_ticker.apply(lambda x: spread_df.point.loc[spread_df.ticker == x].iloc[0])
tp_data['sum_gap'] = ((tp_data['spread'] + tp_data['deviation']) * tp_data['point']) / abs(tp_data['state_open'] - tp_data['state_stop_loss'] ) + 1
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [244]:
x = tp_data.state_chain.iloc[0][0]

tp_data.loc[(tp_data.state_chain[0] == 1)]

KeyError: 'False: boolean label can not be used without a boolean index'

In [247]:
tp_data['state_chain_new'] = [np.array(y) / x for x,y in tp_data[['sum_gap' , 'state_chain']].to_numpy()]

In [250]:
tp_data.state_close_estimate.sum()

46.89175083298029

In [248]:
tp_data

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,spread,deviation,point,sum_gap,state_chain_new
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552...",7,5,0.01,1.571429,"[1.0181818181817892, 1.9878787878787578, 2.260..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]",7,5,0.01,1.413793,"[-0.4048780487804681, -0.6243902439024211]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307...",7,5,0.01,1.923077,"[0.5119999999999807, 0.39199999999998075, 0.27..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0....",7,5,0.01,1.625000,"[-0.4743589743589509, -0.2820512820512558, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0...",7,5,0.01,1.535714,"[0.069767441860467, -0.279069767441868, -0.046..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NFLX,2,2022-11-29 15:30:00,2022-11-30 11:15:00,281.168,283.248,-1.000000,2.354808,-1.000000,-1.000000,-1.000000,"[0.14326923076923273, 0.3596153846153828, 0.29...",24,5,0.01,1.139423,"[0.12573839662447417, 0.31561181434598967, 0.2..."
376,NFLX,1,2022-11-30 14:35:00,2022-11-30 14:40:00,283.605,281.780,-1.000000,-0.523288,-1.000000,-1.000000,-1.000000,"[-0.5232876712329024, -0.9178082191780942]",24,5,0.01,1.158904,"[-0.4515366430260266, -0.791962174940908]"
377,NFLX,1,2022-12-01 20:15:00,2022-12-01 20:20:00,318.462,317.632,-1.000000,-0.592771,-1.000000,-1.000000,-1.000000,"[-0.677108433734967, -0.5927710843373147]",24,5,0.01,1.349398,"[-0.501785714285732, -0.43928571428568647]"
378,NFLX,1,2022-12-02 20:25:00,2022-12-02 20:35:00,320.746,320.056,-1.000000,0.223188,-1.000000,-1.000000,-1.000000,"[0.22318840579701454, -0.09565217391308861, -0...",24,5,0.01,1.420290,"[0.1571428571427958, -0.06734693877554192, -0...."


In [157]:
path =  '/Users/tranthong/Desktop/stock-app/stock_data_M5/'
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = path + ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

start = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_start.iloc[0]
end = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_end.iloc[0]
ticker = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_ticker.iloc[0]
open = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_open.iloc[0]
stoploss = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_stop_loss.iloc[0]
r = abs(open -stoploss)
data =  get_mt5_raw_data_range(ticker, start, end)

In [158]:
(data.Close - open) / r

0       0.292308
1       0.523077
2      -0.015385
3       0.446154
4       0.984615
         ...    
297    18.215385
298    18.215385
299    18.215385
300    18.061538
301    17.138462
Name: Close, Length: 302, dtype: float64

# schedule


In [4]:
import schedule

In [6]:
from datetime import datetime
datetime.now()

datetime.datetime(2023, 3, 9, 16, 57, 32, 391872)